<a href="https://colab.research.google.com/github/An-Ano-nymus/PROJECTS/blob/main/SEV_CV_Colab_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SEV-CV: Self-Evolutionary Generative Transformers (TensorFlow, Colab)


This Colab trains a minimal SEV-CV model with an evolutionary controller on CIFAR-10 or COCO 2017 (subset).

### How to run in Colab (order)
1) Setup: verify TensorFlow/TFDS versions.
2) Mount Google Drive and set DATA_DIR.
3) Prepare datasets into Drive (TFDS or unzip your own images).
4) Define models (Generator/Discriminator).
5) Define data loaders (CIFAR‑10, COCO‑2017, or a Drive folder of images).
6) Define evolution + training utilities.
7) Sanity check: quick forward pass to confirm shapes.
8) Train: pick dataset = 'cifar10' | 'coco2017' | 'folder', adjust img, batch, steps; run training.

Tips:
- COCO uses a subset split by default for speed; expand it if you have space/time.
- All datasets are cached under `/content/drive/MyDrive/SEV-CV/datasets`. Change this if needed.

In [1]:
# Setup: TensorFlow + TFDS (Colab-friendly)
import sys, subprocess, pkgutil

# Optional pin for Colab stability; comment out if you want runtime default
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'tensorflow>=2.12,<2.17', 'tensorflow-datasets>=4.9'])

import tensorflow as tf
import tensorflow_datasets as tfds
print('TF version:', tf.__version__)
print('TFDS version:', tfds.__version__)

TF version: 2.19.0
TFDS version: 4.9.9


## Mount Google Drive and set dataset folder

In [2]:
# Mount Google Drive and set dataset folder
from google.colab import drive
import os

# Mount Google Drive to access files stored there
drive.mount('/content/drive')
# Customize the folder name if you like
DATA_DIR = '/content/drive/MyDrive/SEV-CV/datasets'
# Create the directory if it doesn't exist
os.makedirs(DATA_DIR, exist_ok=True)
print('DATA_DIR =', DATA_DIR)
print('Free space (approx):')
# Check the free space on Google Drive
!df -h /content/drive | tail -n 1

Mounted at /content/drive
DATA_DIR = /content/drive/MyDrive/SEV-CV/datasets
Free space (approx):
drive           108G   43G   66G  40% /content/drive


In [3]:
# Option A: Use TFDS to download directly into Drive (recommended)
import tensorflow_datasets as tfds
import tensorflow as tf
import os

# Define the directory for TensorFlow Datasets within the DATA_DIR
tfds_dir = os.path.join(DATA_DIR, 'tfds')
os.makedirs(tfds_dir, exist_ok=True)
print('TFDS dir:', tfds_dir)

# Example: download CIFAR-10 and COCO 2017 subsets
# Load CIFAR-10 training split
_ = tfds.load('cifar10', data_dir=tfds_dir, split='train', with_info=False)
# Load a 10% subset of COCO 2017 training data
_ = tfds.load('coco/2017', data_dir=tfds_dir, split='train[0%:10%]', with_info=False)
print('TFDS prepared under', tfds_dir)

# Option B: If you already have a zip/tar on Drive, unzip into DATA_DIR
# import zipfile
# zip_path = '/content/drive/MyDrive/path/to/your_dataset.zip'
# with zipfile.ZipFile(zip_path, 'r') as zf:
#     zf.extractall(DATA_DIR)

TFDS dir: /content/drive/MyDrive/SEV-CV/datasets/tfds
TFDS prepared under /content/drive/MyDrive/SEV-CV/datasets/tfds


In [4]:
# Models: SEVGenerator and SEVDiscriminator (minimal)
import tensorflow as tf
import keras
from keras import layers

class MLPBlock(layers.Layer):
    def __init__(self, dim, mlp_ratio=4.0, drop=0.0):
        super().__init__()
        hidden = int(dim * mlp_ratio)
        self.fc1 = layers.Dense(hidden)
        self.act = layers.Activation(tf.nn.gelu)
        self.fc2 = layers.Dense(dim)
        self.drop = layers.Dropout(drop)
    def call(self, x, training=False):
        h = self.fc1(x)
        h = self.act(h)
        h = self.drop(h, training=training)
        h = self.fc2(h)
        h = self.drop(h, training=training)
        return h

class WindowAttention(layers.Layer):
    def __init__(self, dim, heads):
        super().__init__()
        self.attn = layers.MultiHeadAttention(num_heads=heads, key_dim=dim // heads)
        self.nq = layers.LayerNormalization(epsilon=1e-6)
        self.nk = layers.LayerNormalization(epsilon=1e-6)
    def call(self, x, training=False):
        q = self.nq(x)
        k = self.nk(x)
        return self.attn(q, k, training=training)

class TransformerBlock(layers.Layer):
    def __init__(self, dim, heads, mlp_ratio=4.0, drop=0.0):
        super().__init__()
        self.n1 = layers.LayerNormalization(epsilon=1e-6)
        self.attn = WindowAttention(dim, heads)
        self.drop = layers.Dropout(drop)
        self.n2 = layers.LayerNormalization(epsilon=1e-6)
        self.mlp = MLPBlock(dim, mlp_ratio, drop)
    def call(self, x, training=False):
        h = self.attn(self.n1(x), training=training)
        x = x + self.drop(h, training=training)
        h = self.mlp(self.n2(x), training=training)
        x = x + self.drop(h, training=training)
        return x

class PatchUpsample(layers.Layer):
    def __init__(self, out_ch):
        super().__init__()
        self.conv = layers.Conv2DTranspose(out_ch, 4, 2, padding="same")
        self.act = layers.Activation(tf.nn.gelu)
    def call(self, x, training=False):
        return self.act(self.conv(x))

class SEVGenerator(keras.Model):
    def __init__(self, latent_dim=128, base_dim=256, img_size=32, channels=3, depth=4, heads=4):
        super().__init__()
        if img_size % 4 != 0:
            raise ValueError("img_size must be divisible by 4")
        start = img_size // 4
        self.img_size = img_size
        self.fc = layers.Dense(start * start * base_dim)
        self.reshape = layers.Reshape((start, start, base_dim))
        self.up1 = PatchUpsample(base_dim // 2)
        self.up2 = PatchUpsample(base_dim // 4)
        self.to_tokens = layers.Lambda(lambda x: tf.reshape(x, [tf.shape(x)[0], -1, tf.shape(x)[-1]]))
        self.blocks = [TransformerBlock(base_dim // 4, heads) for _ in range(depth)]
        self.to_feat = layers.Lambda(lambda x: tf.reshape(x, [tf.shape(x)[0], img_size, img_size, tf.shape(x)[-1]]))
        self.out = layers.Conv2D(channels, 1, padding="same", activation="tanh")
    def call(self, z, training=False):
        h = self.fc(z)
        h = self.reshape(h)
        h = self.up1(h, training=training)
        h = self.up2(h, training=training)
        t = self.to_tokens(h)
        for b in self.blocks:
            t = b(t, training=training)
        f = self.to_feat(t)
        return self.out(f)

class SEVDiscriminator(keras.Model):
    def __init__(self, img_size=32, channels=3, base=64, heads=4, depth=2):
        super().__init__()
        self.stem = keras.Sequential([
            layers.Conv2D(base, 4, 2, padding="same"), layers.LeakyReLU(0.2),
            layers.Conv2D(base*2, 4, 2, padding="same"), layers.LeakyReLU(0.2),
        ])
        self.flatten_hw = layers.Lambda(lambda x: tf.reshape(x, [tf.shape(x)[0], -1, tf.shape(x)[-1]]))
        dim = base*2
        self.blocks = [TransformerBlock(dim, heads) for _ in range(depth)]
        self.pool = layers.GlobalAveragePooling1D()
        self.out = layers.Dense(1)
    def call(self, x, training=False):
        h = self.stem(x, training=training)
        t = self.flatten_hw(h)
        for b in self.blocks:
            t = b(t, training=training)
        h = self.pool(t)
        return self.out(h)

def build_g(latent_dim=128, img=32, ch=3):
    return SEVGenerator(latent_dim=latent_dim, img_size=img, channels=ch)

def build_d(img=32, ch=3):
    return SEVDiscriminator(img_size=img, channels=ch)

In [5]:
# Data: CIFAR-10 and COCO2017 (subset)
import tensorflow_datasets as tfds
import tensorflow as tf
import os

def _preprocess(ex, img_size):
    x = tf.image.convert_image_dtype(ex['image'], tf.float32)
    h = tf.shape(x)[0]
    w = tf.shape(x)[1]
    side = tf.minimum(h, w)
    x = tf.image.resize_with_crop_or_pad(x, side, side)
    x = tf.image.resize(x, [img_size, img_size], method='bilinear')
    x = x * 2.0 - 1.0
    return x

def make_cifar10(batch=64, img=32, split='train', data_dir=None,
                 shuffle_buf=5000, parallel_calls=2, prefetch=2):
    ds = tfds.load('cifar10', split=split, as_supervised=False, shuffle_files=True, data_dir=data_dir)
    ds = ds.map(lambda e: _preprocess(e, img), num_parallel_calls=parallel_calls)
    ds = ds.shuffle(shuffle_buf).batch(batch, drop_remainder=True)
    return ds.prefetch(prefetch)


TFDS_DIR = '/content/tfds'
def make_coco2017(batch=16, img=64, split='train[0%:10%]', data_dir=TFDS_DIR,
                  shuffle_buf=256, parallel_calls=1, prefetch=1):
    ds = tfds.load('coco/2017', split=split, as_supervised=False, shuffle_files=True, data_dir=data_dir)
    ds = ds.map(lambda e: _preprocess(e, img), num_parallel_calls=parallel_calls)
    ds = ds.shuffle(shuffle_buf).batch(batch, drop_remainder=True)
    return ds.prefetch(prefetch)

# Optional: Load from a folder of images in Drive
def _load_image_file(path, img_size):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    h = tf.shape(img)[0]
    w = tf.shape(img)[1]
    side = tf.minimum(h, w)
    img = tf.image.resize_with_crop_or_pad(img, side, side)
    img = tf.image.resize(img, [img_size, img_size], method='bilinear')
    img = img * 2.0 - 1.0
    return img

def make_folder_dataset(folder, batch=32, img=128, shuffle_buf=2000, parallel_calls=2, prefetch=2):
    files = tf.data.Dataset.list_files(os.path.join(folder, '**', '*.*'), shuffle=True)
    ds = files.map(lambda p: _load_image_file(p, img), num_parallel_calls=parallel_calls)
    ds = ds.shuffle(shuffle_buf).batch(batch, drop_remainder=True)
    return ds.prefetch(prefetch)

In [6]:
# Evolution controller + training utils
import dataclasses, random
import tensorflow as tf
from typing import List

# Dataclass to represent an individual in the population
@dataclasses.dataclass
class Individual:
    lr: float
    heads: int
    depth: int
    score: float = 0.0

# Evolution Controller to manage the population and evolution process
class EvolutionController:
    def __init__(self, population=4, seed=0):
        random.seed(seed)
        self.population: List[Individual] = []
        # Initialize the population with random hyperparameters
        for _ in range(population):
            self.population.append(Individual(lr=1e-4*10**random.uniform(-0.5,0.5), heads=random.choice([2,4,6]), depth=random.choice([2,4,6])))
    # Mutate an individual's hyperparameters
    def mutate(self, ind: Individual) -> Individual:
        return Individual(
            lr=max(1e-5, min(5e-4, ind.lr * (1.0 + random.uniform(-0.3,0.3)))),
            heads=max(2, min(8, ind.heads + random.choice([-2,0,2]))),
            depth=max(2, min(8, ind.depth + random.choice([-2,0,2]))),
        )
    # Select the best individuals and mutate them to form the next generation
    def select(self, k=2):
        self.population.sort(key=lambda i: i.score, reverse=True)
        self.population = self.population[:k] + [self.mutate(self.population[i%k]) for i in range(k, len(self.population))]

# Losses for the GAN
@tf.function
def d_loss_fn(real_logits, fake_logits):
    # Hinge loss for the discriminator
    return tf.reduce_mean(tf.nn.relu(1.0 - real_logits)) + tf.reduce_mean(tf.nn.relu(1.0 + fake_logits))

@tf.function
def g_loss_fn(fake_logits):
    # Hinge loss for the generator
    return -tf.reduce_mean(fake_logits)

# Hinge GAN model
class HingeGAN:
    def __init__(self, img=32, z=128, heads=4, depth=4, lr=2e-4):
        # Initialize Generator and Discriminator models
        self.G = SEVGenerator(latent_dim=z, img_size=img, channels=3, depth=depth, heads=heads)
        self.D = SEVDiscriminator(img_size=img, channels=3, heads=heads, depth=2)
        # Initialize optimizers
        self.opt_g = keras.optimizers.Adam(lr, beta_1=0.0, beta_2=0.99)
        self.opt_d = keras.optimizers.Adam(lr, beta_1=0.0, beta_2=0.99)
        self.z = z
    # Training step for one batch
    @tf.function
    def step(self, real):
        b = tf.shape(real)[0]
        # Generate noise
        noise = tf.random.normal([b, self.z])
        # Train Discriminator
        with tf.GradientTape() as td:
            fake = self.G(noise, training=True)
            r = self.D(real, training=True)
            f = self.D(fake, training=True)
            dl = d_loss_fn(r, f)
        dvars = self.D.trainable_variables
        dgrads = td.gradient(dl, dvars)
        self.opt_d.apply_gradients(zip(dgrads, dvars))

        # Train Generator
        noise = tf.random.normal([b, self.z])
        with tf.GradientTape() as tg:
            fake = self.G(noise, training=True)
            f = self.D(fake, training=True)
            gl = g_loss_fn(f)
        gvars = self.G.trainable_variables
        ggrads = tg.gradient(gl, gvars)
        self.opt_g.apply_gradients(zip(ggrads, gvars))
        return dl, gl

    # Sample images and calculate a simple fitness score (standard deviation of pixel values)
    def sample_fitness(self, n=4):
        z = tf.random.normal([n, self.z])
        imgs = self.G(z, training=False)
        # simple proxy: encourage variance
        return float(tf.math.reduce_std(imgs).numpy())

In [8]:
# =========================================
# 🔧 Training Loop with Checkpoint + Best Model
# =========================================
import time, os, tensorflow as tf

checkpoint_dir = "checkpoints"
best_model_dir = "best_model"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(best_model_dir, exist_ok=True)

# Options: 'cifar10', 'coco2017', or 'folder'
dataset = 'coco2017'  # change as needed
low_mem = True  # reduce RAM on Colab

# Defaults
img = 32 if dataset=='cifar10' else 128
batch = 64 if dataset=='cifar10' else 32
steps = 200

# Low-memory overrides
if low_mem:
    if dataset=='coco2017':
        img = 64
        batch = 8
    else:
        batch = 32
    steps = 100

TFDS_DIR = os.path.join(DATA_DIR, 'tfds') if 'DATA_DIR' in globals() else None
LOCAL_TFDS_DIR = globals().get('LOCAL_TFDS_DIR', None)

make_kwargs = dict(shuffle_buf=1000, parallel_calls=1, prefetch=1) if low_mem else {}

def make_ds_with_fallback():
    try:
        if dataset=='cifar10':
            return make_cifar10(batch=batch, img=img, split='train', data_dir=TFDS_DIR, **make_kwargs)
        elif dataset=='coco2017':
            split = 'train[0%:5%]' if low_mem else 'train[0%:10%]'
            return make_coco2017(batch=batch, img=img, split=split, data_dir=TFDS_DIR, **make_kwargs)
        else:
            custom_folder = os.path.join(DATA_DIR, 'custom_images')
            os.makedirs(custom_folder, exist_ok=True)
            return make_folder_dataset(custom_folder, batch=batch, img=img, **make_kwargs)
    except Exception as e:
        print('Drive-backed TFDS failed, falling back to local cache. Error:', e)
        if dataset=='cifar10':
            return make_cifar10(batch=batch, img=img, split='train', data_dir=LOCAL_TFDS_DIR, **make_kwargs)
        elif dataset=='coco2017':
            split = 'train[0%:5%]' if low_mem else 'train[0%:10%]'
            return make_coco2017(batch=batch, img=img, split=split, data_dir=LOCAL_TFDS_DIR, **make_kwargs)
        else:
            return make_folder_dataset(custom_folder, batch=batch, img=img, **make_kwargs)

ds = make_ds_with_fallback()
it = iter(ds)
evo = EvolutionController(population=2 if low_mem else 4, seed=42)

# Smaller models for low_mem
def build_g_cfg(heads, depth):
    if low_mem:
        return SEVGenerator(latent_dim=96, img_size=img, channels=3, depth=max(2, depth//2), heads=max(2, heads//2), base_dim=192)
    return SEVGenerator(latent_dim=128, img_size=img, channels=3, depth=depth, heads=heads)

def build_d_cfg(heads):
    if low_mem:
        return SEVDiscriminator(img_size=img, channels=3, heads=max(2, heads//2), depth=1, base=48)
    return SEVDiscriminator(img_size=img, channels=3, heads=heads, depth=2)

# Init models
models = []
for ind in evo.population:
    gan = HingeGAN(img=img, z=96 if low_mem else 128, heads=ind.heads, depth=ind.depth, lr=ind.lr)
    gan.G = build_g_cfg(ind.heads, ind.depth)
    gan.D = build_d_cfg(ind.heads)
    models.append(gan)

# ✅ Track best generator loss
best_gen_loss = float("inf")
best_ckpt_path = None

# =========================================
# Training loop with checkpointing
# =========================================
for step in range(1, steps+1):
    try:
        real = next(it)
    except Exception as e:
        print('Iterator failed, recreating dataset/iterator. Error:', e)
        ds = make_ds_with_fallback()
        it = iter(ds)
        real = next(it)

    for i, gan in enumerate(models):
        dl, gl = gan.step(real)

        # Check if this generator is best so far
        if gl < best_gen_loss:
            best_gen_loss = gl
            best_ckpt_path = os.path.join(best_model_dir, f"best_gan_{i}_step{step}")
            gan.G.save(best_ckpt_path + "_G.keras")
            gan.D.save(best_ckpt_path + "_D.keras")
            print(f"🌟 New best generator (loss={gl:.4f}) saved at step {step} → {best_ckpt_path}")

    # Step-based checkpointing
    if step % 20 == 0:
        for i, gan in enumerate(models):
            g_path = os.path.join(checkpoint_dir, f"step{step}_gan{i}_G.keras")
            d_path = os.path.join(checkpoint_dir, f"step{step}_gan{i}_D.keras")
            gan.G.save(g_path)
            gan.D.save(d_path)
        print(f"✅ Checkpoints saved at step {step}")

    # Evolution step
    if step % 10 == 0:
        for i, (gan, ind) in enumerate(zip(models, evo.population)):
            ind.score = gan.sample_fitness(n=2 if low_mem else 4)
        evo.select(k=1 if low_mem else 2)

        # rebuild population
        models = []
        for ind in evo.population:
            new_gan = HingeGAN(img=img, z=96 if low_mem else 128, heads=ind.heads, depth=ind.depth, lr=ind.lr)
            new_gan.G = build_g_cfg(ind.heads, ind.depth)
            new_gan.D = build_d_cfg(ind.heads)
            models.append(new_gan)

        print(f"[step {step}] evolved heads/depth/lr:", [(ind.heads, ind.depth, round(ind.lr,6)) for ind in evo.population])

print('🎯 Training complete.')
print(f"✅ Best generator loss observed: {best_gen_loss:.4f} at {best_ckpt_path}")


Iterator failed, recreating dataset/iterator. Error: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} /content/drive/MyDrive/SEV-CV/datasets/tfds/coco/2017/1.1.0/coco-train.tfrecord-00012-of-00256; Transport endpoint is not connected [Op:IteratorGetNext] name: 


Drive-backed TFDS failed, falling back to local cache. Error: [Errno 107] Transport endpoint is not connected: '/content/drive/MyDrive'


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

ResourceExhaustedError: /root/tensorflow_datasets/coco/2017/incomplete.XKXWNR_1.1.0/bucket_ecc313e9-d00d-47bd-af7d-50f1bebd3748_884.tmp; No space left on device

In [ ]:
# =========================================
# 🔍 Visualization after Preprocessing
# =========================================
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

# Pick a batch of data from your dataset iterator
batch_images = next(iter(ds))

# -----------------------------------------
# 1. Show sample preprocessed images
# -----------------------------------------
num_images = min(9, batch_images.shape[0])  # only take up to 9
plt.figure(figsize=(10, 6))
for i in range(num_images):
    plt.subplot(3, 3, i + 1)
    plt.imshow(((batch_images[i].numpy() + 1) / 2).clip(0, 1))  # scale back to [0,1]
    plt.axis("off")
plt.suptitle("Sample Preprocessed Images", fontsize=16)
plt.show()


# -----------------------------------------
# 2. Pixel intensity distribution (histogram)
# -----------------------------------------
flat_pixels = batch_images.numpy().flatten()
plt.figure(figsize=(8, 5))
sns.histplot(flat_pixels, bins=50, kde=True)
plt.title("Pixel Intensity Distribution after Preprocessing")
plt.xlabel("Pixel value [-1, 1]")
plt.ylabel("Frequency")
plt.show()

# -----------------------------------------
# 3. Correlation heatmap between color channels
# -----------------------------------------
# Reshape: (batch, H*W, channels)
pixels = tf.reshape(batch_images, [-1, 3]).numpy()
channel_corr = tf.convert_to_tensor(pixels).numpy()
corr_matrix = sns.heatmap(
    data=tf.cast(tf.transpose(channel_corr), tf.float32).numpy() @ tf.cast(channel_corr, tf.float32).numpy(),
    cmap="coolwarm", center=0
)
plt.title("Heatmap: Correlation between RGB Channels")
plt.show()

# -----------------------------------------
# 4. Variance Heatmap of a single image
# -----------------------------------------
img = batch_images[0].numpy()
# Compute variance across channels
var_map = img.var(axis=-1)
plt.figure(figsize=(6, 6))
sns.heatmap(var_map, cmap="mako")
plt.title("Variance Heatmap (per pixel across RGB)")
plt.axis("off")
plt.show()

# -----------------------------------------
# 5. Feature visualization with FFT (frequency domain)
# -----------------------------------------
from numpy.fft import fft2, fftshift
gray = tf.image.rgb_to_grayscale((img + 1) / 2).numpy().squeeze()
fft_img = np.log(1 + np.abs(fftshift(fft2(gray))))

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(gray, cmap="gray")
plt.title("Grayscale Image")
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(fft_img, cmap="inferno")
plt.title("FFT (Frequency Spectrum)")
plt.axis("off")
plt.show()


In [ ]:
# Save generator weights
save_dir = '/content/drive/MyDrive/SEV-CV/models'
os.makedirs(save_dir, exist_ok=True)
G_test.save_weights(os.path.join(save_dir, 'sev_generator.h5'))
print("Generator weights saved.")


In [ ]:
# # Sanity check: forward generator at chosen resolution
# img_test = 128
# # Build a Generator model for testing at a specific resolution
# G_test = SEVGenerator(latent_dim=128, img_size=img_test, channels=3, depth=2, heads=4)
# # Generate a batch of random noise and pass it through the generator
# y = G_test(tf.random.normal([2,128]), training=False)
# # Print the shape of the generator's output
# print('gen out shape:', y.shape)


#version 2 for low memory:
# Sanity check: forward generator at chosen resolution
z_dim = 96 if low_mem else 128
img_test = 128

G_test = SEVGenerator(latent_dim=z_dim, img_size=img_test, channels=3, depth=2, heads=4)
y = G_test(tf.random.normal([2, z_dim]), training=False)
print('✅ Generator sanity check output shape:', y.shape)



In [ ]:
# Rebuild model with same architecture
G_loaded = SEVGenerator(latent_dim=latent_test, img_size=img_test, channels=3, depth=2, heads=4)
G_loaded.load_weights(os.path.join(save_dir, 'sev_generator.h5'))
print("Generator weights loaded.")


In [ ]:
z = tf.random.normal([4, latent_test])
gen_imgs = G_loaded(z, training=False)

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.imshow(((gen_imgs[i].numpy() + 1) / 2).clip(0,1))
    plt.axis("off")
plt.suptitle("Generated Images from Saved Model")
plt.show()


**Reasoning**:
Generate sample images using the first trained generator model.

